In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from itertools import product
import warnings

# for neural networks
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

# for evaluation & preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
)
import sys, os

sys.path.append(os.path.abspath('..'))
%load_ext autoreload
%autoreload 2
from modules.config import *
from modules.storage import (
    get_demand_model_data,
    store_results
)
from modules.evaluation import get_evaluation_metrics
from modules.neural_network import *

ModuleNotFoundError: No module named 'keras'

In [ ]:
def train_model(
    X_train, y_train, batch_size, nodes_per_feature, n_layers, activation, dropout
):
    model = Sequential()
    model.add(
        Dense(nodes_per_feature, activation=activation, input_shape=(X_train.shape[1],))
    )
    n_features = X_train.shape[1]
    for _ in range(n_layers):

        model.add(Dense(nodes_per_feature * n_features, activation=activation))
        if dropout >= 0:
            model.add(Dropout(dropout))
    model.add(Dense(1, activation="relu"))

    model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])

    early_stopping = EarlyStopping(patience=5, min_delta=0.001)
    n_epochs = 40
    history = model.fit(
        X_train,
        y_train,
        epochs=n_epochs,
        batch_size=batch_size,
        validation_split=0.25,
        callbacks=[early_stopping],
        verbose=0,
    )
    n_trained_epochs = len(history.history["loss"])
    if n_trained_epochs == n_epochs:
        warnings.warn("Model was stopped while validation loss was still improving")
    return model


In [ ]:
def execute_stage(
    results_path, get_hyperparameters, h3_res, time_interval_length, test_phase=False, 
):
    model_data = get_demand_model_data(h3_res, time_interval_length)
    model_data = model_data.iloc[:1000]
    X_train, X_valid, X_test, y_train, y_valid, y_test = split_and_scale_data(
        model_data,
        "demand"
    )
    if test_phase:
        X_train = np.concatenate([X_train, X_valid])
        y_train = np.concatenate([y_train, y_valid])

        X_valid = X_test
        y_valid = y_test

    for model_params in tqdm(get_hyperparameters()):
        console_out = (
            f"batch_size: {model_params['batch_size']} - "
            + f"nodes_per_feature: {model_params['nodes_per_feature']} - "
            + f"n_layers: {model_params['n_layers']} - "
            + f"activation: {model_params['activation']} - "
            + f"dropout: {model_params['dropout']}"
        )
        tqdm.write(console_out, end="\r")
        if check_if_model_result_empty(
            results_path, h3_res, time_interval_length, model_params
        ):
            tqdm.write(console_out + " # already trained")
            continue

        model = train_model(
            X_train,
            y_train,
            model_params["batch_size"],
            model_params["nodes_per_feature"],
            model_params["n_layers"],
            model_params["activation"],
            model_params["dropout"],
        )
        tqdm.write(console_out + " # trained", end="\r")
        y_pred_for_validation = model.predict(X_valid)

        results = {
            "h3_res": h3_res,
            "time_interval_length": time_interval_length,
            "batch_size": model_params["batch_size"],
            "nodes_per_feature": model_params["nodes_per_feature"],
            "n_layers": model_params["n_layers"],
            "activation": model_params["activation"],
            "dropout": model_params["dropout"],
            **get_evaluation_metrics(
                y_valid, y_pred_for_validation, "test" if test_phase else "val"
            ),
        }
        store_results(pd.DataFrame(data=results, index=[0]), results_path)
        tqdm.write(console_out + " # evaluated")


In [ ]:
execute_stage(
    NN_FIRST_STAGE_RESULTS_PATH,
    get_first_stage_hyperparameters,
    TUNE_H3_RESOLUTION,
    TUNE_TIME_INTERVAL_LENGTH,
)

  0%|          | 0/4 [00:00<?, ?it/s]

batch_size: 32 - nodes_per_feature: 1 - n_layers: 1 - activation: relu - dropout: -1 # already trained
batch_size: 64 - nodes_per_feature: 1 - n_layers: 1 - activation: relu - dropout: -1 # already trained
batch_size: 128 - nodes_per_feature: 1 - n_layers: 1 - activation: relu - dropout: -1 # already trained
batch_size: 256 - nodes_per_feature: 1 - n_layers: 1 - activation: relu - dropout: -1 # already trained


In [ ]:
results = get_results_df(NN_FIRST_STAGE_RESULTS_PATH)

best_batch_size = (
    results[
        (results["h3_res"] == TUNE_H3_RESOLUTION)
        & (results["time_interval_length"] == TUNE_TIME_INTERVAL_LENGTH)
    ]
    .sort_values(by="val_mape", ascending=True)["batch_size"]
    .iloc[0]
)

first_stage_hyperparameters = get_first_stage_hyperparameters()
batch_sizes = list(map(lambda x: x['batch_size'], first_stage_hyperparameters))
max_batch_size = max(batch_sizes)
min_batch_size = min(batch_sizes)

print(f"best batch_size: **{best_batch_size}** - min: {min_batch_size} - max: {max_batch_size}")


best batch_size: **256** - min: 32 - max: 256


In [ ]:
get_hyperparameters = lambda : get_second_stage_hyperparameters(best_batch_size)
execute_stage(
    NN_SECOND_STAGE_RESULTS_PATH,
    get_hyperparameters,
    TUNE_H3_RESOLUTION,
    TUNE_TIME_INTERVAL_LENGTH,
)

  0%|          | 0/27 [00:00<?, ?it/s]

batch_size: 256 - nodes_per_feature: 0.5 - n_layers: 1 - activation: relu - dropout: -1 # already trained
batch_size: 256 - nodes_per_feature: 0.5 - n_layers: 1 - activation: sigmoid - dropout: -1 # already trained
batch_size: 256 - nodes_per_feature: 0.5 - n_layers: 1 - activation: tanh - dropout: -1 # already trained
batch_size: 256 - nodes_per_feature: 0.5 - n_layers: 2 - activation: relu - dropout: -1 # already trained
batch_size: 256 - nodes_per_feature: 0.5 - n_layers: 2 - activation: sigmoid - dropout: -1 # already trained
batch_size: 256 - nodes_per_feature: 0.5 - n_layers: 2 - activation: tanh - dropout: -1 # already trained
batch_size: 256 - nodes_per_feature: 0.5 - n_layers: 3 - activation: relu - dropout: -1 # already trained
batch_size: 256 - nodes_per_feature: 0.5 - n_layers: 3 - activation: sigmoid - dropout: -1 # already trained
batch_size: 256 - nodes_per_feature: 0.5 - n_layers: 3 - activation: tanh - dropout: -1 # already trained
batch_size: 256 - nodes_per_feature: 

In [ ]:
results = get_results_df(NN_SECOND_STAGE_RESULTS_PATH)
best_model = (
    results[
        (results["h3_res"] == TUNE_H3_RESOLUTION)
        & (results["time_interval_length"] == TUNE_TIME_INTERVAL_LENGTH)
    ]
    .sort_values(by="val_mape", ascending=True)
    .iloc[0]
)

best_config = {
    "batch_size": best_model["batch_size"],
    "nodes_per_feature": best_model["nodes_per_feature"],
    "n_layers": best_model["n_layers"],
    "activation": best_model["activation"],
}

best_config


{'batch_size': 256,
 'nodes_per_feature': 1.5,
 'n_layers': 2,
 'activation': 'relu'}

In [ ]:
get_hyperparameters = lambda : get_third_stage_hyperparameters(
    best_batch_size=best_model["batch_size"],
    best_nodes_per_feature=best_model["nodes_per_feature"],
    best_n_layers=best_model["n_layers"],
    best_activation=best_model["activation"],
)
execute_stage(
    NN_THIRD_STAGE_RESULTS_PATH,
    get_hyperparameters,
    TUNE_H3_RESOLUTION,
    TUNE_TIME_INTERVAL_LENGTH,
)


  0%|          | 0/4 [00:00<?, ?it/s]

batch_size: 256 - nodes_per_feature: 1.5 - n_layers: 2 - activation: relu - dropout: 0 # already trained
batch_size: 256 - nodes_per_feature: 1.5 - n_layers: 2 - activation: relu - dropout: 0.05 # already trained
batch_size: 256 - nodes_per_feature: 1.5 - n_layers: 2 - activation: relu - dropout: 0.1 # already trained
batch_size: 256 - nodes_per_feature: 1.5 - n_layers: 2 - activation: relu - dropout: 0.2 # already trained


In [ ]:
results = get_results_df(NN_THIRD_STAGE_RESULTS_PATH)
best_dropout = (
	results[
		(results["h3_res"] == TUNE_H3_RESOLUTION)
		& (results["time_interval_length"] == TUNE_TIME_INTERVAL_LENGTH)
	]
	.sort_values(by="val_mape", ascending=True)["dropout"]
	.iloc[0]
)
best_config = {
	**best_config,
	"dropout": best_dropout,
}
best_config
		

{'batch_size': 256,
 'nodes_per_feature': 1.5,
 'n_layers': 2,
 'activation': 'relu',
 'dropout': 0.05}

In [ ]:
for h3_res, time_interval_length in product(CALC_H3_RESOLUTIONS, CALC_TIME_INTERVAL_LENGTHS):
	print(h3_res, time_interval_length)
	execute_stage(
		NN_FOURTH_STAGE_RESULTS_PATH,
		lambda : [best_config],
		h3_res,
		time_interval_length,
		test_phase=True,
	)

7 1


  0%|          | 0/1 [00:00<?, ?it/s]

batch_size: 256 - nodes_per_feature: 1.5 - n_layers: 2 - activation: relu - dropout: 0.05 # already trained
7 2


  0%|          | 0/1 [00:00<?, ?it/s]

batch_size: 256 - nodes_per_feature: 1.5 - n_layers: 2 - activation: relu - dropout: 0.05 # already trained
7 6
7 24


...

batch_size: 256 - nodes_per_feature: 1.5 - n_layers: 2 - activation: relu - dropout: 0.05 # evaluated
9 2


  0%|          | 0/1 [00:00<?, ?it/s]

batch_size: 256 - nodes_per_feature: 1.5 - n_layers: 2 - activation: relu - dropout: 0.05 # evaluated
9 6
9 24


  0%|          | 0/1 [00:00<?, ?it/s]

batch_size: 256 - nodes_per_feature: 1.5 - n_layers: 2 - activation: relu - dropout: 0.05 # evaluated


In [ ]:
results = get_results_df(NN_FOURTH_STAGE_RESULTS_PATH)
results

,h3_res,time_interval_length,batch_size,nodes_per_feature,n_layers,activation,dropout,val_mse,val_mae,val_mape,val_rmse,test_mse,test_mae,test_mape,test_rmse
0,7,1,256,1.5,2,relu,0.05,None,None,None,None,0.801164,0.661631,0.438166,0.895078
1,7,2,256,1.5,2,relu,0.05,None,None,None,None,2.238201,1.110350,0.520476,1.496062
2,7,24,256,1.5,2,relu,0.05,None,None,None,None,239.377734,11.813052,0.859131,15.471837
3,8,6,256,1.5,2,relu,0.05,None,None,None,None,2.270000,1.323333,1.000000,1.506652
4,9,1,256,1.5,2,relu,0.05,None,None,None,None,0.109484,0.217080,0.194982,0.330884
5,9,2,256,1.5,2,relu,0.05,None,None,None,None,1.463333,1.136667,1.000000,1.209683
6,9,24,256,1.5,2,relu,0.05,None,None,None,None,0.349757,0.326694,0.264173,0.591403
